In [348]:
import pandas as pd
from balance_generation import simulate_balance_by_account_level
from interest_rate_generation import simulate_interest_rate
from shutil import copy

In [362]:
neo4j_path = '/Users/gianluca/neo4j-enterprise-5.26.12/'
dataset = "sf0.01"
records_df = pd.DataFrame()
if dataset == "sf0.01":
    records_df = pd.read_csv(f'{neo4j_path}import/sf0.01/snapshot/Account.csv', sep='|')[:2000]
elif dataset == "sf0.1":
    records_df = pd.read_csv(f'{neo4j_path}import/sf0.1/snapshot/Account.csv', sep='|')[:4000]
elif dataset == "sf0.3":
    records_df = pd.read_csv(f'{neo4j_path}import/sf0.3/snapshot/Account.csv', sep='|')[:6000]
elif dataset == "sf1":
    records_df = pd.read_csv(f'{neo4j_path}import/sf1/snapshot/Account.csv', sep='|')[:10000]

records_df['createTime'] = pd.to_datetime(records_df['createTime'].astype(str), format='mixed', errors='coerce').dt.date

results = []
for _, row in records_df.iterrows():
    ts = simulate_balance_by_account_level(level=row['accountLevel'], start_date=row['createTime'])
    # Extract timestamps and values as separate lists
    timestamps = [item[0].strftime("%Y-%m-%dT%H:%M:%S") if item[0].tzinfo else item[0].strftime("%Y-%m-%dT%H:%M:%S") + "+0000" for item in ts]

    # Only convert to float if item[1] is a number, otherwise keep as is
    values = [round(float(item[1]),3) if isinstance(item[1], (int, float)) else item[1] for item in ts]
    results.append({
        'id': row['accountId'],
        'balance_timestamps': timestamps,
        'balance_values': values
    })

# Convert results to DataFrame and write to CSV
results_df = pd.DataFrame(results)
results_df.to_csv('account_balances.csv', index=False)
print(len(results_df))
# Copy the CSV file to the desired location
copy('account_balances.csv', f'{neo4j_path}import/{dataset}/snapshot/account_balances.csv')



10000


'/Users/gianluca/neo4j-enterprise-5.26.12/import/sf1/snapshot/account_balances.csv'